In [2]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)
import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])

# Import the backtrader platform
import backtrader as bt
import pandas as pd
import backtrader.strategies as st
from backtrader_plotting import Bokeh
import backtrader.indicators as btind

cerebro = bt.Cerebro(exactbars=1, optreturn=False)

# Create a Data Feed
data = bt.feeds.GenericCSVData(
dataname='ETH.csv',
dtformat='%Y-%m-%dT%H:%M:%SZ',
fromdate=datetime.datetime(2018, 1, 1),
datetime=0,
open=1,
high=2,
low=3,
close=4,
volume=-1,
openinterest=-1
)

cerebro.adddata(data)
# Add strategy to Cerebro
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe_ratio')

# Create a Stratey
class MA_CrossOver(bt.Strategy):
    
    alias = ('SMA_CrossOver',)

    params = (
        # period for the fast Moving Average
        ('fast', 10),
        # period for the slow moving average
        ('slow', 30),
        # moving average to use
        ('_movav', btind.MovAv.SMA)
    )

    def __init__(self):
        sma_fast = self.p._movav(period=self.p.fast)
        sma_slow = self.p._movav(period=self.p.slow)

        self.buysig = btind.CrossOver(sma_fast, sma_slow)

    def next(self):
        if self.position.size:
            if self.buysig < 0:
                self.sell()

        elif self.buysig > 0:
            self.buy()

cerebro.optstrategy(
    MA_CrossOver, fast=range(5, 10), slow=range(50, 60)
) 
###################################################################################################
if __name__ == '__main__':
    
    # Add a strategy
    cerebro.addstrategy(MA_CrossOver)

    # Default position size
    cerebro.addsizer(bt.sizers.AllInSizer)

    if __name__ == '__main__':

        optimized_runs = cerebro.run()

        final_results_list = []
        # Iterate through list of lists
        for run in optimized_runs:
            for strategy in run:
                PnL = round(strategy.broker.get_value(), 2)
                sharpe = strategy.analyzers.sharpe_ratio.get_analysis()
                final_results_list.append(
                    [
                        strategy.params.fast,
                        strategy.params.slow,
                        PnL,
                        sharpe['sharperatio'],
                    ]
                )

    sort_by_sharpe = sorted(final_results_list, key=lambda x: x[2], reverse=True)
    # Print top 5 results sorted by Sharpe Ratio
    for line in sort_by_sharpe[:5]:
        print(line)
    # Set our desired cash start
    cerebro.broker.setcash(1000.0)

    # Print out the starting conditions
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Print out the final result
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

    #Run over everything
    cerebro.run()
    b = Bokeh(style='bar', plot_mode='single')
    cerebro.plot(b)